# Step 1: Install packages

In [1]:
%pip install langchain langgraph langchain-google-genai python-dotenv

# Step 2: Get APIKEY (GEMINI)

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY")

# Step 3: Create agent

I have done:

* Define a simple weather tool (get_weather)

* Initialize the Gemini model (gemini-2.5-flash-lite)

* Create an agent using create_agent

* Invoke the agent with a user message

In [3]:
from dotenv import load_dotenv
load_dotenv()

from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI

# Tool simple
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

# Modelo Gemini
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.5
)

# Crear agente
agent = create_agent(
    model=model,
    tools=[get_weather],
    system_prompt="You are a helpful assistant"
)

# Ejecutar
response = agent.invoke(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]}
)

print(response)

{'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='3f67fa70-016c-4cb1-957a-dd30f66d8884'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "SF"}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c8359-47f6-7941-84b0-8cc2a37b2926-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'SF'}, 'id': '27ecb2b1-ed25-4bd1-863a-d1c6199fcfc3', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 52, 'output_tokens': 15, 'total_tokens': 67, 'input_token_details': {'cache_read': 0}}), ToolMessage(content="It's always sunny in SF!", name='get_weather', id='49030502-c79e-4d28-9ccc-9aa35c3c99e9', tool_call_id='27ecb2b1-ed25-4bd1-863a-d1c6199fcfc3'), AIMessage(content="The weather in SF is: It's always sunny in SF!", additional_kwargs={}, response_me

# Step 4: System prompt

This is a custom SYSTEM_PROMPT that:

* Defines the assistant personality (pun-based weather forecaster)

* Specifies available tools

* Instructs how to decide when to use each tool

The system prompt controls:

* Agent reasoning style

* Tool selection behavior

* Decision-making constraints

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
SYSTEM_PROMPT = """You are an expert weather forecaster who speaks in puns.

You have access to two tools:

- get_weather_for_location
- get_user_location

If a user asks for the weather, make sure you know the location.
If they mean where they are, use get_user_location.
"""

# Step 5: Create tools
I define:

1. get_weather_for_location

Returns weather for a specific city.

2. get_user_location

Uses a runtime context object to determine the user’s location.

To enable structured runtime context, allows user-specific tool behavior, demonstrates dependency injection into tools.

In [6]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@dataclass
class Context:
    user_id: str

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Get the user's location based on their user_id in the runtime context."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "San Francisco"

# Step 6: Configure model

I initialize the model

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.5
)

# Step 7: Response format
This cell does:

* Enforces structured output.

* Makes responses predictable.

* Useful for API integration and production systems.

In [8]:
@dataclass
class ResponseFormat:
    punny_response: str
    weather_conditions: str | None = None

# Step 8: Adding memory

This cell does:

* Enables multi-turn conversation.

* Persists state per thread_id.

* Allows the agent to remember previous messages.

In [9]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

# Step 9: Create agent

I build the fully configured agent with:

* Model

* System prompt

* Tools

* Context schema

* Structured output strategy

* Memory checkpointer

This integrates all previous components into one cohesive system.

In [10]:
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)

# Step 10: Execute agent

I invoke the agent with:

* messages

* config containing a thread_id

* context containing a user_id

So the:

* thread_id enables memory persistence.

* context allows personalized tool behavior.

* Structured response ensures predictable output.

In [11]:
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "What is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response["structured_response"])

ResponseFormat(punny_response="Seems like you're in for some fun in the sun! Don't forget your shades, it's going to be a bright one!", weather_conditions='Sunny')


# Step 11: Test memory

In [12]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "Thank you!"}]},
    config=config,
    context=Context(user_id="1")
)

print(response["structured_response"])

ResponseFormat(punny_response="No need to be um-believably grateful! I'm always here to help you weather any storm.", weather_conditions=None)
